In [3]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import re
import json
from russtress import Accent
from pyphonetics import Soundex
from fonetika.soundex import RussianSoundex
import rusyllab
from fonetika.distance import PhoneticsInnerLanguageDistance
from collections import defaultdict
import nltk

In [16]:
p = re.compile(r'[а-яА-ЯёЁ]+')
lines = []
with open('Lermontov all (raw text).txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip().split()) > 2:
            lines.append(line.strip())

In [17]:
lines[:1000]

['Листья в поле пожелтели,',
 'И кружатся, и летят;',
 'Лишь в бору поникши ели',
 'Зелень мрачную хранят.',
 'Под нависшею скалою',
 'Уж не любит, меж цветов,',
 'Пахарь отдыхать порою',
 'От полуденных трудов.',
 'Зверь, отважный, поневоле',
 'Скрыться где-нибудь спешит.',
 'Ночью месяц тускл и поле',
 'Сквозь туман лишь серебрит.',
 'Однажды женщины Эрота отодрали;',
 'Досадой раздражен, упрямое дитя,',
 'Напрягши грозный лук и за обиду мстя,',
 'Не смея к женщинам, к нам ярость острой стали,',
 'Не слушая мольбы усерднейшей, стремит.',
 'Ваш подлый род один! — безумный говорит.',
 'С тех пор-то женщина любви не знает!,.',
 'И точно как рабов считает нас она...',
 'Так в паказаниях всегда почти бывает:',
 'Которые смирней, на тех падет вина!..',
 'На склоне гор, близ вод, прохожий, зрел ли ты',
 'Беседку тайную, где грустные мечты',
 'Сидят задумавшись? Над ними свод акаций:',
 'Там некогда стоял алтарь п муз и граций,',
 'И куст прелестных роз, взлелеяпных веспой.',
 'Там некогда, 

In [18]:
accent = Accent()
soundex = RussianSoundex(delete_first_letter=True, code_vowels=True)
phon_distance = PhoneticsInnerLanguageDistance(soundex)

In [19]:
def get_rhyme_ending(word):
    stress_pos = accent.put_stress(word).find('\'')
    if stress_pos == -1:
        return word

    lst = list(word)
    lst[stress_pos-1] = lst[stress_pos-1].upper()
    word = ''.join(lst)
    sx = rusyllab.split_words([word])
    for i in range(len(sx)):
        if not sx[i].islower():
            return ''.join(sx[i:]).lower()

def is_rhyme(word1, word2):
    end1 = get_rhyme_ending(word1)
    end2 = get_rhyme_ending(word2)
    if phon_distance.distance(end1, end2) <= 1:
        return True
    else:
        return False

def check_rhyme_end_with_word(end, word):
    word_end = get_rhyme_ending(word)
    if phon_distance.distance(end, word_end) <= 1:
        return True
    else:
        return False

In [20]:
word = 'мая'

In [25]:
import string

In [44]:
rhyme_lines = []
for i, line in enumerate(lines):
    if i % 100 == 0:
        print(i)
    s = line.translate(str.maketrans('', '', string.punctuation))
#     if len(s.split()) < 0:
#         print(s.split)
    if len(s) > 1:
        try:
            if is_rhyme(s.split()[-1], word):
                rhyme_lines.append(line)
        except IndexError:
            print(line)

0
100
200
300
400
500
600
700
800
900
1000
* * *
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600


KeyboardInterrupt: 

In [47]:
d = {'a':[1], 'b':[2]}
d.setdefault('c', [3])

[3]

In [48]:
d

{'a': [1], 'b': [2], 'c': [3]}

In [51]:
d.setdefault('d', []).append(4)

In [52]:
d

{'a': [1, [3]], 'b': [2], 'c': [3], 'd': [4]}

In [61]:
rhymes_dict = {}
p = re.compile(r'[а-яА-ЯёЁ]+')
for i, line in enumerate(lines):
    if i % 100 == 0:
        print(i)
    s = line.translate(str.maketrans('', '', string.punctuation))
    try:
        last_word = p.findall(s.split()[-1])
        if last_word:
            rhymes_dict.setdefault(soundex.transform(get_rhyme_ending(last_word[0])), []).append(line)
    except IndexError:
        print(line)

0
100
200
300
400
500
600
700
800
900
1000
* * *
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18

In [62]:
len(rhymes_dict)

3791

In [63]:
with open('rhymes_dict_sdx.txt', 'w') as f:
    f.write(json.dumps(rhymes_dict))

In [59]:
p = re.compile(r'[а-яА-ЯёЁ]+')
for i, ending in enumerate(rhymes_dict):
    print(p.findall(ending.strip()))

['тели']
['тят']
['ели']
['нят']
['лою']
['тов']
['рою']
['дов']
['воле']
['шит']
['поле']
['рит']
['рали']
['тя']
['мстя']
['стали']
['мит']
['знает']
['на']
['вает']
['ты']
['каций']
['граций']
['пой']
['ной']
['вой']
['ривый']
['бовь']
['кровь']
['маны']
['ляны']
['мне']
['не']
['мннанье']
['данье']
['лог']
['рок']
['ницей']
['венный']
['щенный']
['чал']
['хищенный']
['пал']
['десный']
['дой']
['мой']
['бесный']
['нет']
['ет']
['лиры']
['тдшине']
['сне']
['миры']
['ланит']
['ненья']
['жит']
['нит']
['петерсон']
['денья']
['ренья']
['звон']
['станет']
['узрпт']
['манет']
['бросить']
['ти']
['косит']
['падет']
['лодны']
['ет']
['родной']
['чать']
['тые']
['дать']
['вые']
['россии']
['дей']
['змии']
['зей']
['япной']
['той']
['ваный']
['кои']
['ваюсь']
['о']
[]
['ряюсь']
[]
['музы']
['дар']
['узы']
['жар']
['суден']
['завлек']
['труден']
['сенья']
['тел']
['взором']
['ей']
['чей']
['вором']
['друг']
['суг']
['ца']
['ливой']
['пиво']
['феба']
['хлеба']
['випа']
['франции']
['ницы']
['ня

['воли']
['душу']
['правый']
['трудно']
['плачет']
['вица']
['таман']
['хладен']
['любит']
['щитных']
['верить']
['мерить']
['лыбки']
['шибки']
['рен']
['хладный']
['радный']
['тиг']
['лений']
['дом']
['лах']
['хал']
['звал']
['хом']
['мчался']
['вечер']
['видно']
['жарко']
['смутно']
['торый']
['заньем']
['лям']
['хожий']
['этот']
['раве']
['гонек']
['тивном']
['мост']
['клязьма']
['хав']
['ближе']
['дор']
['б']
['твердо']
['мглав']
['роги']
['капли']
['за']
['вежит']
['это']
['ше']
['меет']
['бу']
['чай']
['жизнью']
['заться']
['лыбка']
['книги']
['чины']
['луке']
['больше']
['рывки']
['этим']
['ялся']
['или']
['любим']
['лестью']
['верной']
['телин']
['мим']
['дила']
['леньям']
['веньем']
['шеньем']
['хих']
['пять']
[]
['павел']
['тавил']
['валерьян']
['пьян']
['кеев']
['теличеев']
['рат']
['брат']
['чина']
['судьбина']
['дана']
['рокая']
['е']
['рой']
['свила']
['ятное']
['милый']
['ночп']
['ворны']
['корны']
['лась']
['силась']
['дами']
['женства']
['шенства']
['властью']
['счасть

['мшистой']
['нистой']
['бине']
['судьбине']
['рялось']
['щалось']
['нылой']
['жасной']
['дерева']
['ряла']
['хала']
['всплыла']
['ристой']
['пилой']
['латный']
['ратный']
['рожным']
['нежным']
['тежным']
['чалив']
['ленпой']
['менной']
['окой']
['еть']
['кнули']
['роса']
['степь']
['изгнанных']
['цепь']
['встала']
['вились']
['тились']
['говая']
['невших']
['злобной']
['добный']
['пет']
['шумный']
['клубпл']
['молвный']
['тьме']
['скалам']
['токой']
['дишь']
['ратишь']
[]
[]
[]
[]
['димых']
['бимых']
['ятствовало']
['смелым']
['ливым']
['п']
[]
['настья']
['цаньем']
['стал']
['боком']
['соком']
['своду']
['вестной']
['лестной']
['места']
['кладбищу']
['сырым']
['чавый']
['брег']
['бег']
['нится']
['лился']
['частной']
['блестя']
['востока']
[]
['рокий']
['скалами']
['домчится']
['тропами']
['афоса']
['лемоса']
['островами']
['вспоминанье']
['времен']
['страданье']
['точенный']
['хую']
['далую']
['ать']
['бега']
['хая']
['лало']
['токих']
['брегами']
['наться']
['ваем']
['пас']
['мельк

['хапку']
['шапку']
['бавной']
['равный']
['нались']
['мей']
['драмы']
['дамы']
['падком']
['рядком']
['лани']
['бранп']
['тата']
['зримый']
['дома']
['славы']
['луг']
['буду']
['нуя']
['лупы']
['бодренный']
['редно']
['взяли']
['сын']
['изгнанный']
['отнес']
['пес']
['гались']
['стуча']
['корана']
['пзгнанья']
['юта']
['нута']
['маза']
['львица']
['хребте']
['вели']
['ветных']
['тений']
['роил']
['ил']
['чадрой']
['зыбкой']
['росою']
['гудала']
['быни']
['синодала']
['вырезной']
['ложена']
['шелками']
['карабаха']
['тизны']
['Лер']
['снежной']
['боге']
['битву']
['тьмою']
['папах']
['ствол']
['зины']
['плит']
['мест']
['рудной']
['трудной']
['сетина']
['паленный']
['ханный']
['жатье']
['кара']
['тамара']
['ласный']
['встанет']
['взглянет']
['денницы']
['пуг']
['ноновый']
['нитель']
['вест']
['спаситель']
['сеня']
['лекли']
['свеч']
['храма']
['фимиама']
['щельи']
['кельи']
['птиц']
['авроры']
['дымок']
['цины']
['снеговой']
['гучий']
['сонной']
['коной']
['стонет']
['гонит']
['ноком']

In [64]:
import edlib

In [66]:
edlib.align('подъезд', 'падъезд')["editDistance"]

1

In [67]:
get_rhyme_ending('неваляшка')

'ляшка'

In [68]:
get_rhyme_ending('говняшка')

'няшка'

In [69]:
soundex.transform('ляшка')

'7A53A'

In [70]:
soundex.transform('няшка')

'8A53A'

In [71]:
phon_distance.distance('ляшка', 'няшка')

1

In [72]:
sdx_ending = soundex.transform(get_rhyme_ending(word))

In [73]:
sdx_ending

'8JA'

In [81]:
rhyme_lines = []
sdx_ending = soundex.transform(get_rhyme_ending(word))
for i, end in enumerate(rhymes_dict):
    if i % 100 == 0:
        print(i)
    if edlib.align(end, sdx_ending)["editDistance"] < 1:
        rhyme_lines += rhymes_dict[end]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700


In [89]:
random.choice(rhyme_lines)

'Чтоб хоть одна рука родная'

In [100]:
with open('rhymes_dict_sdx.txt', 'r') as f:
    rhymes_dict1 = json.loads(f.read())

In [101]:
rhymes_dict1

{'4B7B': ['Листья в поле пожелтели,',
  'И снова муки посетили',
  'Не раз они в Сион ходили,',
  'Чего б они мне не простили.',
  'Что люди угасить в душе моей хотели',
  'В лесу холодном в грозный час метели',
  'Тогда б мазурки опустели.',
  'Поверю ль я, чтоб вы хотели',
  'Три года ровно вы шутили',
  'Они у ней вырвать хотели.',
  'Всё, что вы прежде возбудили',
  'Едва дымятся фитили;',
  'Как при Ермолове ходили',
  'Ручей телами запрудили.',
  'На мягкой пуховой постели,',
  'чтоб его пощадили.',
  'Мы часто на берег сходили',
  'И часто по степям бродили,',
  'Лучи вечерние златили',
  'Как паруса вдали бродили.',
  'Его приверженцы хотели',
  'И мысли зреньем овладели...',
  'Стоят! Он странен, в самом деле!',
  'Красавицы раздумавшись сидели,',
  'Унылый свист играющей метели.',
  'Нескромный месяц дымкою одели,',
  'Читал большую книгу, что кадили,',
  '(Не помню я, в который день недели),—',
  'А Саша всё один был на постели.',
  'Скажу короче: в две недели',
  'Когда она

In [95]:
def get_rhyme_line(sent):
    rhyme_lines = []
    p = re.compile(r'[а-яА-ЯёЁ]+')
    word = p.findall(sent)[-1]
    sdx_ending = soundex.transform(get_rhyme_ending(word))
    for end in rhymes_dict:
        if edlib.align(end, sdx_ending)["editDistance"] < 1:
            rhyme_lines += rhymes_dict[end]
    if rhyme_lines:
        return random.choice(rhyme_lines)
    else
        return 'Рифма тут так себе будет...'

In [99]:
get_rhyme_line('люблю грозу в начале мая')

'И пробегут они, хромая,'

In [102]:
random.choice([])

IndexError: Cannot choose from an empty sequence